# Interactive Session 1: Exploring Neural Network Fundamentals

## Welcome to Interactive Brain Modeling!

This interactive session will guide you through the fundamental concepts of neural networks and their relationship to brain function. You'll experiment with different parameters and see their effects in real-time.

### Session Overview
- Interactive visualization of single neurons
- Parameter exploration with sliders and widgets
- Real-time plotting of activation functions
- Hands-on exploration of learning dynamics

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🧠 Welcome to Interactive Brain Modeling! 🧠")
print("Let's explore how artificial neurons work...")

## Interactive Element 1: Neuron Activation Functions

Use the sliders below to explore different activation functions and their parameters!

In [ ]:
def plot_activation_function(func_type='sigmoid', steepness=1.0, threshold=0.0):
    """Interactive plot of activation functions"""
    x = np.linspace(-5, 5, 100)
    
    if func_type == 'sigmoid':
        y = 1 / (1 + np.exp(-steepness * (x - threshold)))
        title = f'Sigmoid Function (steepness={steepness:.1f}, threshold={threshold:.1f})'
    elif func_type == 'tanh':
        y = np.tanh(steepness * (x - threshold))
        title = f'Tanh Function (steepness={steepness:.1f}, threshold={threshold:.1f})'
    elif func_type == 'relu':
        y = np.maximum(0, steepness * (x - threshold))
        title = f'ReLU Function (slope={steepness:.1f}, threshold={threshold:.1f})'
    else:  # step function
        y = np.where(x > threshold, 1, 0)
        title = f'Step Function (threshold={threshold:.1f})'
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, linewidth=3, label=func_type.capitalize())
    plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    plt.xlabel('Input (x)', fontsize=12)
    plt.ylabel('Output (activation)', fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(-1.1, 1.1)
    plt.show()

# Create interactive widgets
function_widget = widgets.Dropdown(
    options=['sigmoid', 'tanh', 'relu', 'step'],
    value='sigmoid',
    description='Function:'
)

steepness_widget = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Steepness:'
)

threshold_widget = widgets.FloatSlider(
    value=0.0,
    min=-3.0,
    max=3.0,
    step=0.1,
    description='Threshold:'
)

# Create interactive plot
interactive_plot = widgets.interactive(
    plot_activation_function,
    func_type=function_widget,
    steepness=steepness_widget,
    threshold=threshold_widget
)

display(interactive_plot)

## Interactive Element 2: Simple Perceptron Decision Boundary

Watch how a perceptron learns to classify data points!

In [ ]:
def interactive_perceptron_learning(learning_rate=0.1, num_points=50, noise_level=0.1):
    """Interactive visualization of perceptron learning"""
    
    # Generate random data
    np.random.seed(42)
    X = np.random.randn(num_points, 2)
    # Create linearly separable data with some noise
    y = (X[:, 0] + X[:, 1] > 0).astype(int) * 2 - 1  # -1 or 1
    
    # Add noise
    noise_indices = np.random.choice(len(y), int(noise_level * len(y)), replace=False)
    y[noise_indices] *= -1
    
    # Simple perceptron learning
    weights = np.random.randn(3) * 0.1  # Including bias
    X_with_bias = np.column_stack([np.ones(len(X)), X])
    
    # Plot initial state
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot data points
    colors = ['red' if label == -1 else 'blue' for label in y]
    ax1.scatter(X[:, 0], X[:, 1], c=colors, s=50, alpha=0.7)
    ax1.set_xlabel('Feature 1')
    ax1.set_ylabel('Feature 2')
    ax1.set_title('Perceptron Classification (Initial)')
    ax1.grid(True, alpha=0.3)
    
    # Plot decision boundary (initial)
    if weights[1] != 0:  # Avoid division by zero
        x_line = np.linspace(X[:, 0].min()-1, X[:, 0].max()+1, 100)
        y_line = -(weights[0] + weights[1] * x_line) / weights[2]
        ax1.plot(x_line, y_line, 'k--', linewidth=2, label='Initial Decision Boundary')
    ax1.legend()
    ax1.set_xlim(X[:, 0].min()-1, X[:, 0].max()+1)
    ax1.set_ylim(X[:, 1].min()-1, X[:, 1].max()+1)
    
    # Learning process
    errors = []
    for epoch in range(100):
        epoch_errors = 0
        for i in range(len(X)):
            prediction = np.sign(np.dot(X_with_bias[i], weights))
            if prediction != y[i]:
                weights += learning_rate * y[i] * X_with_bias[i]
                epoch_errors += 1
        errors.append(epoch_errors)
        if epoch_errors == 0:
            break
    
    # Plot final decision boundary
    if weights[1] != 0:
        x_line = np.linspace(X[:, 0].min()-1, X[:, 0].max()+1, 100)
        y_line = -(weights[0] + weights[1] * x_line) / weights[2]
        ax1.plot(x_line, y_line, 'g-', linewidth=3, label='Final Decision Boundary')
    ax1.legend()
    
    # Plot learning curve
    ax2.plot(errors, 'o-', linewidth=2, markersize=4)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Number of Errors')
    ax2.set_title('Learning Progress')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Converged after {len(errors)} epochs!")
    print(f"Final weights: {weights}")

# Create interactive controls
lr_widget = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Learning Rate:')
points_widget = widgets.IntSlider(value=50, min=20, max=200, step=10, description='Num Points:')
noise_widget = widgets.FloatSlider(value=0.1, min=0.0, max=0.5, step=0.05, description='Noise Level:')

interactive_perceptron = widgets.interactive(
    interactive_perceptron_learning,
    learning_rate=lr_widget,
    num_points=points_widget,
    noise_level=noise_widget
)

display(interactive_perceptron)

## Discussion Questions

After exploring the interactive elements above, consider these questions:

1. **Activation Functions**: How do different activation functions relate to the "all-or-nothing" firing behavior of biological neurons?

2. **Learning Dynamics**: What happens when you increase the learning rate? Why might very high learning rates be problematic?

3. **Noise and Robustness**: How does the perceptron handle noisy data? What does this tell us about the robustness of simple neural models?

4. **Biological Inspiration**: Which aspects of these artificial neurons seem most similar to biological neurons? Which aspects are clearly different?

## Next Steps

In the next interactive session, we'll explore more complex architectures and their biological inspirations!